In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

CUDA Available? True


In [3]:
# Train data transformations
train_transforms = transforms.Compose([
    transforms.RandomApply([transforms.CenterCrop(22), ], p=0.1),
    transforms.Resize((28, 28)),
    transforms.RandomRotation((-15., 15.), fill=0),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
    ])

# Test data transformations
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])

In [4]:
train_data = datasets.MNIST('../data', train=True, download=True, transform=train_transforms)
test_data = datasets.MNIST('../data', train=False, download=True, transform=test_transforms)

100%|██████████| 9912422/9912422 [00:00<00:00, 142223788.66it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 104788662.48it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 238199745.01it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 4557542.77it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.dropout = nn.Dropout(0.1)
        # Block 1
        self.conv1 = nn.Conv2d(1, 128, 3, padding=1, bias=True)  #28>>28 | 1>>3 | 1>>1
        self.norm1 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 8, 1, padding=1, bias=True) #28>>30 | 3>>3 | 1>>1
        self.conv2 = nn.Conv2d(8, 16, 3, stride=1, padding=1, bias=True) # 30>>30 | 3>>5 | 1>>1
        self.norm2 = nn.BatchNorm2d(16)
        #self.conv3 = nn.Conv2d(16, 16, 3, stride=1, padding=1, bias=False) #30>>30 | 5>>7 | 1>>1
        #self.norm3 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2) #30>>15 | 5>>6 | 1>>2
        # Block 2
        self.conv5 = nn.Conv2d(16, 16, 3, stride=1, padding=1, bias=True) #15>> 15 | 6>>10 | 2>>2
        self.norm5 = nn.BatchNorm2d(16)
        self.conv6 = nn.Conv2d(16, 32, 3, stride=1, padding=1, bias=True) #15>>15 | 10>>14 | 2>>2
        self.norm6 = nn.BatchNorm2d(32)
        self.conv7 = nn.Conv2d(32, 16, 1, stride=1, padding=1, bias=True) #15>>17 | 14>>14 | 2>>2
        self.pool2 = nn.MaxPool2d(2, 2) #17>>8 | 14>>16 | 2>>4
        # Block 3
        self.conv8 = nn.Conv2d(16, 16, 3, stride=1, padding=1, bias=True) #8>> 8 | 16>>24 | 4>>4
        self.norm8 = nn.BatchNorm2d(16)
        self.conv9 = nn.Conv2d(16, 32, 3, stride=1, padding=1, bias=True) #8>> 8 | 24>>32 | 4>>4
        self.norm9 = nn.BatchNorm2d(32)
        self.conv10 = nn.Conv2d(32, 1, 1, stride=1, padding=1, bias=True) #8>>10 | 32>>32 | 4>>4
        self.fc1 = nn.Linear(100, 10, bias=True)
        #self.gpool = nn.AvgPool2d(kernel_size=7)
        #self.fc1 = nn.Linear(128, 10, bias=False)
        #self.fc2 = nn.Linear(50, 10, bias=False)
        #self.conv6 = nn.Conv2d(512, 1024, 3, bias=False)
        #self.conv7 = nn.Conv2d(1024, 10, 3, bias=False)

    def forward(self, x):
        #x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        #x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        #x = F.relu(self.conv6(F.relu(self.conv5(x))))
        #x = F.relu(self.conv7(x))
        #x = x.view(-1, 10)
        x = F.relu(self.conv1(x), 2)
        x = self.dropout(self.norm1(x))
        x = self.conv4(x)
        x = F.relu(self.conv2(x),2)
        x = self.dropout(self.norm2(x))
        #x = F.relu(self.conv3(x),2)
        #x = self.dropout(self.norm3(x))
        x = self.pool1(x)

        x = F.relu(self.conv5(x), 2)
        x = self.dropout(self.norm5(x))
        x = F.relu(self.conv6(x), 2)
        x = self.dropout(self.norm6(x))
        x = self.conv7(x)
        x = self.pool2(x)

        x = F.relu(self.conv8(x), 2)
        x = self.dropout(self.norm8(x))
        x = F.relu(self.conv9(x), 2)
        x = self.dropout(self.norm9(x))
        x = self.conv10(x)
        #x = self.norm4(x)
        #print('SIze', x.size)
        #x = x.view(-1, 3200)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)

        #x = self.gpool(x)
        #x = x.view(-1, 10)
        #x = F.relu(self.fc1(x))
        #x = self.fc2(x)
        return F.log_softmax(x)

In [7]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 128, 28, 28]           1,280
       BatchNorm2d-2          [-1, 128, 28, 28]             256
           Dropout-3          [-1, 128, 28, 28]               0
            Conv2d-4            [-1, 8, 30, 30]           1,032
            Conv2d-5           [-1, 16, 30, 30]           1,168
       BatchNorm2d-6           [-1, 16, 30, 30]              32
           Dropout-7           [-1, 16, 30, 30]               0
         MaxPool2d-8           [-1, 16, 15, 15]               0
            Conv2d-9           [-1, 16, 15, 15]           2,320
      BatchNorm2d-10           [-1, 16, 15, 15]              32
          Dropout-11           [-1, 16, 15, 15]               0
           Conv2d-12           [-1, 32, 15, 15]           4,640
      BatchNorm2d-13           [-1, 32, 15, 15]              64
          Dropout-14           [-1, 32,

<ipython-input-6-b6c143dc8572>:71: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [8]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [12]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        #if batch_idx ==0:
          #print('data', data)
          #print('target:', target)
        optimizer.zero_grad()
        #if batch_idx ==0:
         #print(optimizer)
        output = model(data)
        #if batch_idx ==0:
          #print(output)
        loss = F.nll_loss(output, target)
        #if batch_idx ==0:
          #print('loss', loss)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.1f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [13]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.015, momentum=0.9)
num_epochs = 19
for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-6-b6c143dc8572>:71: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.07610896974802017 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 23.30it/s]



Test set: Average loss: 0.0563, Accuracy: 9807/10000 (98.1%)



loss=0.07460219413042068 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.91it/s]



Test set: Average loss: 0.0379, Accuracy: 9876/10000 (98.8%)



loss=0.025096869096159935 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 24.70it/s]


...


Test set: Average loss: 0.0183, Accuracy: 9941/10000 (99.4%)



loss=0.02105111815035343 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.85it/s]



Test set: Average loss: 0.0182, Accuracy: 9938/10000 (99.4%)



loss=0.003417445346713066 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 24.45it/s]



Test set: Average loss: 0.0179, Accuracy: 9942/10000 (99.4%)

